In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize

from datetime import datetime

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Boolean
from sqlalchemy.orm import Session

In [3]:
# Here are the account names
accounts_list = [
    'burberry',
    'gucci',
    'toryburch',
    'michaelkors',
    'bananarepublic',
    'majeofficiel',
    'aliceandolivia',
    'coach',
    'ferragamo',
    'chloe'
]

In [4]:
# Make 2 dataframes, one for account info, one for posts

# Hold each account's dfs to concat later
account_info_df_list = []
account_posts_dfs_list = []

# Process each account json file
for account in accounts_list:
    
# Load the json file
    account_dict = json.load(open(f'data/appended/appended_{account}.json'))
    
# Store the top-level info in accounts info df

    account_info_df = json_normalize(account_dict)
    # Trim down the dataframe for only essential columns
    account_info_df = account_info_df[[
        'account_name', 'follower_count',
        'following_count', 'post_count'
    ]]
    # Add to list to concat later
    account_info_df_list.append(account_info_df)
    
# Make the account posts df from post data list

    posts_data_list = account_dict['post_data_list']
    account_posts_df = json_normalize(posts_data_list)
    
    # Add a column for account name to use as key later
    account_posts_df['account_name'] = account_dict['account_name']
    
    # Parse the datetime string to save as datetime object
    account_posts_df['post_datetime'] = [
        datetime.strptime(each, '%Y-%m-%dT%H:%M:%S.000Z') \
        for each in account_posts_df['post_datetime_str']
    ]
    # Rearrange columns for easier reading
    account_posts_df = account_posts_df[[
        'account_name', 'post_datetime', 'post_type',
        'like_count', 'view_count',
        'has_hashtag', 'hashtag_count', 'has_at', 'at_count',
        'post_url'
    ]]
    
    # Add to list ot concat later
    account_posts_dfs_list.append(account_posts_df)

In [5]:
# Concat the lists to make 2 dfs
all_accounts_info_df = pd.concat(account_info_df_list).reset_index(drop=True)
all_accounts_posts_df = pd.concat(account_posts_dfs_list).reset_index(drop=True)

In [6]:
# Trim down the post data for between 2014/01/01 and 2018/03/01
date_range = (all_accounts_posts_df['post_datetime'] > datetime(2014, 1, 1, 0, 0)) & (all_accounts_posts_df['post_datetime'] < datetime(2018, 3, 1, 0, 0))
posts_in_date_range = all_accounts_posts_df[date_range].reset_index(drop=True)

In [7]:
posts_in_date_range['hashtag_count'] = posts_in_date_range['hashtag_count'].astype(float)
posts_in_date_range['at_count'] = posts_in_date_range['at_count'].astype(float)

In [8]:
posts_in_date_range

,account_name,post_datetime,post_type,like_count,view_count,has_hashtag,hashtag_count,has_at,at_count,post_url
0,burberry,2018-02-28 20:05:18,photo,29493.0,NaN,True,2.0,True,1.0,https://www.instagram.com/p/BfwM2bqHG__/?taken...
1,burberry,2018-02-28 17:03:12,photo,15140.0,NaN,True,1.0,True,1.0,https://www.instagram.com/p/Bfv4Auon0Pu/?taken...
2,burberry,2018-02-28 15:40:19,photo,32967.0,NaN,True,1.0,True,1.0,https://www.instagram.com/p/BfvuhonAVVK/?taken...
3,burberry,2018-02-28 14:01:58,photo,21800.0,NaN,True,2.0,True,1.0,https://www.instagram.com/p/BfvjRWQAo38/?taken...
4,burberry,2018-02-28 12:47:49,photo,28015.0,NaN,True,2.0,True,1.0,https://www.instagram.com/p/BfvayOogIvM/?taken...
5,burberry,2018-02-28 11:07:12,photo,24644.0,NaN,True,2.0,True,1.0,https://www.instagram.com/p/BfvPRRkAIZq/?taken...
6,burberry,2018-02-27 20:35:35,photo,33956.0,NaN,True,1.0,True,1.0,https://www.instagram.com/p/BftrhYGg-UD/?taken...
7,burberry,2018-02-27 17:09:29,photo,25460.0,NaN,True,2.0,True,1.0,https://www.instagram.com/p/BftT722ggoK/?taken...
8,burberry,2018-02-27 15:26:49,photo,17952.0,NaN,True,1.0,True,1.0,https://www.instagram.com/p/BftIL7wgH-z/?taken...
9,burberry,2018-02-27 13:48:37,photo,37005.0,NaN,True,1.0,True,1.0,https://www.instagram.com/p/Bfs88ogAvNS/?taken...


In [9]:
# Create classes to map tables
Base = declarative_base()

class Account(Base):
    __tablename__ = 'accounts'
    
    id = Column(Integer, primary_key=True)
    account_name = Column(String)
    follower_count = Column(Integer)
    following_count = Column(Integer)
    post_count = Column(Integer)

class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    account_name = Column(String)
    post_datetime = Column(DateTime)
    post_type = Column(String)
    like_count = Column(Integer)
    view_count = Column(Integer)
    has_hashtag = Column(Boolean)
    hashtag_count = Column(Integer)
    has_at = Column(Boolean)
    at_count = Column(Integer)
    post_url = Column(String)

In [10]:
# Begin sqlite portion

# Start it up
engine = create_engine("sqlite:///data/project_db.sqlite")
Base.metadata.create_all(engine)

session = Session(bind=engine)


In [11]:
# Add accounts rows to a list
accounts_rows = []

for i in range(0, len(all_accounts_info_df)):
    row = Account(
        id=i,
        account_name=all_accounts_info_df['account_name'][i],
        follower_count=int(all_accounts_info_df['follower_count'][i]),
        following_count=int(all_accounts_info_df['following_count'][i]),
        post_count=int(all_accounts_info_df['post_count'][i]),
    )
    
    accounts_rows.append(row)

session.add_all(accounts_rows)
session.commit()

In [13]:
# Add posts rows to a list
posts_rows = []

for i in range(0, len(posts_in_date_range)):
    row = Post(
        id=i,
        account_name=posts_in_date_range['account_name'][i],
        post_datetime = posts_in_date_range['post_datetime'][i],
        post_type = posts_in_date_range['post_type'][i],
        like_count = posts_in_date_range['like_count'][i],
        view_count = posts_in_date_range['view_count'][i],
        has_hashtag = posts_in_date_range['has_hashtag'][i],
        hashtag_count = posts_in_date_range['hashtag_count'][i],
        has_at = posts_in_date_range['has_at'][i],
        at_count = posts_in_date_range['at_count'][i],
        post_url = posts_in_date_range['post_url'][i]
    )
    
    posts_rows.append(row)

session.add_all(posts_rows)
session.commit()

In [ ]:
from sqlalchemy import inspect
inst = inspect(Post)
attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
attr_names

In [ ]:
# Save files
all_accounts_info_df.to_csv('data/accounts.csv', index=False)
all_accounts_posts_df.to_csv('data/posts.csv', index=False)

In [ ]:
# Unify the two dataframes
unified_df = all_accounts_posts_df.merge(all_accounts_info_df, 
                      how='left', on='account_name')

In [ ]:
unified_df.to_csv('data/posts_with_account.csv', index=False)